## Choosing hyperparameters for GAiA's neural network

In this notebook, we are trying different numbers of SE-ResNet blocks for our final model.

In [4]:
from model.GAiA_network import GAiA_Network, coeff_determination
from lib.dataset_utils import read_many_hdf5, board_shape

import pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

import platform
if platform.system() == "Darwin":
    %config InlineBackend.figure_format="retina"  # For high DPI display

if tf.test.gpu_device_name(): 
  print(f"Default GPU Device: {tf.test.gpu_device_name()}")

Default GPU Device: /device:GPU:0


2021-12-16 01:30:39.485606: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-16 01:30:39.749923: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-16 01:30:39.928210: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-16 01:30:39.928436: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

We are loading our training dataset and we are using it to build different training and validation data
for each model.

In [ ]:
directory = "/media/gaetan/HDD/IA/Chess/Datasets/SE_ResNet/"
positions, scores = read_many_hdf5(1389333, directory, "_train")
positions = positions[:150000]
scores = scores[:150000]

nb_models = 10
positions_train, scores_train = [], []
positions_valid, scores_valid = [], []
for i in range(nb_models):
  positions_train_t, positions_valid_t, scores_train_t, scores_valid_t = train_test_split(
    positions, scores, test_size=0.2, random_state=42+i)
  positions_train.append(positions_train_t)
  positions_valid.append(positions_valid_t)
  scores_train.append(scores_train_t)
  scores_valid.append(scores_valid_t)
print(f"Training sets shape ≈ {positions_train[0].shape}, Validation sets shape ≈ {positions_valid[0].shape}")


Now, we create several models with an increasing number of SE-ResNet blocks.

In [ ]:
models = []
offset_blocks = 2
filters = [64, 128]
for i in range(1, nb_models+1):
  model = GAiA_Network(board_shape, {"filters":filters[(i-1) // (nb_models//2)], "nb_blocks":i+offset_blocks})
  model.compile(loss="mean_absolute_error", optimizer="adam", metrics=[coeff_determination])
  models.append(model)
models[0].summary()

We train our models.

In [ ]:
nb_epochs = 15
histories = []
for i, model in enumerate(models):
  print(f"Model {i+1}/{len(models)}...", end=" ")
  history = model.fit(positions_train[i], scores_train[i], verbose=0, epochs=nb_epochs)
  histories.append(history)
  pred = model.predict(positions_train[i])
  r2_score = coeff_determination(tf.convert_to_tensor(scores_train[i]), pred).numpy()
  print(f"Done. Training accuracy: {r2_score:.2f}")


In [ ]:
def plot_validation_results(histories, validation_Xs, validation_ys, path=None, ext="pdf"):
  epochs = range(1, nb_epochs + 1)

  fig, axs = plt.subplots(1, 2, figsize=(20, 5))
  _, ax_score  = plt.subplots(1, 1, figsize=(10, 5))

  line_styles = ["-", "--", "-.", ":", ".-"]

  for i, history in enumerate(histories):
    X_valid = validation_Xs[i]
    y_valid = validation_ys[i]
    nb_blocks = i + offset_blocks + 1
    nb_filters = filters[i // (nb_models//2)]

    loss = history.history["loss"]
    score = history.history["coeff_determination"]
    style = line_styles[i % len(line_styles)]
    axs[0].plot(epochs, loss, style, label=f"{nb_blocks} SE-ResNet blocks {nb_filters} filters")

    axs[1].plot(epochs, score, style, label=f"{nb_blocks} SE-ResNet blocks {nb_filters} filters")

    preds = models[i].predict(X_valid)
    r2_score = coeff_determination(tf.convert_to_tensor(y_valid), preds).numpy()
    ax_score.scatter(y_valid, preds, label = f"{nb_blocks} SE-ResNet blocks  {nb_filters} filters $R^2 = {r2_score:.2f}$")


  axs[0].set_xlabel("Epoch")
  axs[0].set_ylabel("(Mean Absolute Error)")
  axs[0].set_title('Training loss')
  axs[1].set_xlabel("Epoch")
  axs[1].set_ylabel("($R^2$)")
  axs[1].set_title('Training score')
  for i, ax in enumerate(axs):
    ax.legend()
    if path:
      extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
      fig.savefig(path + '_' + str(i) + "_1." + ext, bbox_inches=extent)

  ma = np.max(y_valid)
  mi = np.min(y_valid)
  x = np.linspace(ma, mi, 100)
  ax_score.plot(x, x, "-.", label = "Predictions = True values")
  ax_score.set_xlabel("True values (centipawn)")
  ax_score.set_ylabel("Predictions (centipawn)")
  ax_score.set_title("Results on validation set")
  ax_score.legend()

  if path:
    plt.savefig(path + "_2." + ext)

def get_best_model(models, validation_Xs, validation_ys):
  preds = []
  for i in range(len(models)):
    pred = models[i].predict(validation_Xs[i])
    r2_score = coeff_determination(tf.convert_to_tensor(validation_ys[i]), pred).numpy()
    preds.append(r2_score)
  idx = np.argmax(preds)+1
  return models[idx], offset_blocks + idx, filters[idx // (nb_models//2)]

Here are the results!

In [ ]:
plot_validation_results(histories, positions_valid, scores_valid, path="results/model_selection")

We can see on the figure above that all of our model perform well.

However, adding more blocks and filters do not improve the results.

Therefore, we can take a not too complex one.

In [ ]:
config = {"nb_blocks": 4, "filters": 64}
print(config)
output_file = open("model/hyperparameters.pickle", "wb")
pickle.dump(config, output_file)
output_file.close()